In [5]:
#Importing libraries
import pandas as pd
import time
from bs4 import BeautifulSoup as bs
from splinter import Browser
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
from webdriver_manager.chrome import ChromeDriverManager

In [14]:
#Establsih path to chrome executable file
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\jorge\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


## NASA Mars News

[NASA - Mars Exploration Program](https://mars.nasa.gov/news/)




In [15]:
# Visit the NASA Mars News Page
# Set up url
nasa_url = "https://mars.nasa.gov/news/"
browser.visit(nasa_url)

# Wait for the page/page elements to load.
time.sleep(1)

# Scrape page into soup.
html = browser.html
soup = bs(html, "html.parser")

In [16]:
# Get latest news title and paragraph text.
news_title = soup.find_all('div', class_='content_title')[1].text
news_p = soup.find_all('div', class_="article_teaser_body")[0].text
print(f"Latest news title: {news_title}.")
print(f"Latest news p text: {news_p}")

Latest news title: NASA's Ingenuity Helicopter and Asteroid-Monitoring System Honored.
Latest news p text: The team behind the agency's history-making rotorcraft soars, picking up high-profile awards alongside a new-and-improved system to assess asteroid orbits.


In [18]:


# Navigate to the page
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

time.sleep(4)

# Assign the HTML content of the page to a variable
imgs_html = browser.html
# Parse HTML with Beautifulsoup
soup = bs(imgs_html,'html.parser')

# Find the image url for the current Featured Mars Image
img_result = soup.find('article', class_="lede full_width")
print(img_result)
# img_url = img_result.replace("background-image: url('","").replace("');","")
# featured_image_url = f"https://www.jpl.nasa.gov{img_url}"

# print(featured_image_url)



None
